<font color='tomato'><font color="#CC3D3D"><p>
# A Tutorial on WRMF(Weighted Regularized Matrix Factorization)

이 튜토리얼은 아래 github에 구현된 Cornac-compatible WRMF 모델을 최신의 TF 버전으로 수정한 것임: TF 1.x => TF 2.x
- https://github.com/ddoeunn/Weighted-Regularized-Matrix-Factorization

### Setup

In [ ]:
import pandas as pd
import numpy as np

# MS recommenders API 
import sys
sys.path.append('/home/work/yhcho/2023-02/RS')  # 본인이 msr.zip 압축을 푼 위치를 확인(셀에서 pwd 명령어 실행) 후 변경해야 함. 
                                                # 윈도우에서는 폴더 구분자를 // 또는 \\로 해야 함.  
from msr.cornac_utils import predict_ranking
from msr.python_splitters import python_stratified_split

# Cornac API 
import cornac
print(f"Cornac version: {cornac.__version__}")
from cornac.eval_methods import BaseMethod, RatioSplit, StratifiedSplit, CrossValidation
from cornac.metrics import Precision, Recall, NDCG, AUC, MAP
from cornac.data import FeatureModality
from cornac.models import EASE

# Custom models
from recom_wrmf import WRMF

In [ ]:
# Data column definition
DEFAULT_USER_COL = 'resume_seq'
DEFAULT_ITEM_COL = 'recruitment_seq'
DEFAULT_RATING_COL = 'rating'
DEFAULT_PREDICTION_COL = 'prediction'

# Top k items to recommend
TOP_K = 5

# Random seed, Verbose, etc.
SEED = 202311
VERBOSE = True

### Preprocessing

In [ ]:
# 데이터 로딩
data = pd.read_csv('../kmu_comp/apply_train.csv')
data[DEFAULT_RATING_COL] = 1  # Cornac에서 지정한 데이터형식(UIR: User, Item, Rating)에 따라

# 데이터 분할
train, test = python_stratified_split(
    data, 
    filter_by="user", 
    ratio=0.7,
    col_user=DEFAULT_USER_COL, col_item=DEFAULT_ITEM_COL,
    seed=SEED
)

### Modeling

In [ ]:
params = {
    'name': "WRMF-uniform_pos",
    'weight_strategy': "uniform_pos",
    'data': train
}

wrmf_uniform_pos = WRMF(**params, seed=SEED, verbose=VERBOSE)

In [ ]:
params = {
    'name': "WRMF-uniform_neg",
    'weight_strategy': "uniform_neg",
    'data': train
}

wrmf_uniform_neg = WRMF(**params, seed=SEED, verbose=VERBOSE)

In [ ]:
params = {
    'name': "WRMF-user-oriented",
    'weight_strategy': "user_oriented",
    'data': train
}

wrmf_user_oriented = WRMF(**params, seed=SEED, verbose=VERBOSE)

In [ ]:
params = {
    'name': "WRMF-item-oriented",
    'weight_strategy': "item_oriented",
    'data': train
}

wrmf_item_oriented = WRMF(**params, seed=SEED, verbose=VERBOSE)

In [ ]:
params = {
    'name': "WRMF-item-popularity",
    'weight_strategy': "item_popularity",
    'data': train
}

wrmf_item_popularity = WRMF(**params, seed=SEED, verbose=VERBOSE)

### Evaluation

In [ ]:
# 평가방법 설정
eval_method = BaseMethod.from_splits(
    train_data=np.array(train), 
    test_data=np.array(test), 
    exclude_unknowns=True,  # Unknown users and items will be ignored.
    verbose=True
)

# 평가척도 설정
metrics = [Recall(k=TOP_K), NDCG(k=TOP_K)]

# 실험 수행
cornac.Experiment(
    eval_method=eval_method,
    models=[wrmf_uniform_pos, wrmf_uniform_neg, wrmf_user_oriented, wrmf_item_oriented, wrmf_item_popularity],
    metrics=metrics,
).run()

### Deployment

In [ ]:
# 전체 데이터(rating/side-info)로 다시 학습
# data.Dataset.from_uir()이 FeatureModality를 지원하지 않아 .fit()의 파라미터로 item feature를 전달
full_data = cornac.data.Dataset.from_uir(data.itertuples(index=False), seed=SEED)

model = WRMF(weight_strategy="item_popularity", data=data, seed=SEED, verbose=VERBOSE)  
model.fit(full_data)

# 모든 item에 대한 예측값 생성
all_pred = predict_ranking(
    model, data, 
    usercol=DEFAULT_USER_COL, itemcol=DEFAULT_ITEM_COL, 
    remove_seen=True
)

# Top-K item 생성
top_k = (
    all_pred
    .groupby(DEFAULT_USER_COL)
    .apply(lambda x: x.nlargest(TOP_K, DEFAULT_PREDICTION_COL))
    .reset_index(drop=True)
    .drop(DEFAULT_PREDICTION_COL, axis=1)
    .sort_values(by=DEFAULT_USER_COL)
)

# submission 화일 저장
t = pd.Timestamp.now()
fname = f"submit_{model.name}_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
top_k.to_csv(fname, index=False)

<font color='tomato'><font color="#CC3D3D"><p>
# End